In [120]:
#import necessary library
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
import logging
# Load the data
data = pd.read_csv('stores_sales.csv',
                   parse_dates=['date']
                   )
display(data)

<ipython-input-120-3fbf2aa2838e>:11: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  data = pd.read_csv('stores_sales.csv',


,store,date,weekly_sales,holiday_flag,temperature,fuel_Price,cpi,unemployment
0,1,2010-05-02,1643690.90,0,42.31,2.572,211.096358,8.106
1,1,2010-12-02,1641957.44,1,38.51,2.548,211.242170,8.106
2,1,2010-02-19,1611968.17,0,39.93,2.514,211.289143,8.106
3,1,2010-02-26,1409727.59,0,46.63,2.561,211.319643,8.106
4,1,2010-05-03,1554806.68,0,46.50,2.625,211.350143,8.106
...,...,...,...,...,...,...,...,...
6430,45,2012-09-28,713173.95,0,64.88,3.997,192.013558,8.684
6431,45,2012-05-10,733455.07,0,64.89,3.985,192.170412,8.667
6432,45,2012-12-10,734464.36,0,54.47,4.000,192.327265,8.667
6433,45,2012-10-19,718125.53,0,56.47,3.969,192.330854,8.667


## Data Preprocessing

In [121]:
# Check missing values per column
missing_values = data.isnull().sum()
print(missing_values)

store           0
date            0
weekly_sales    0
holiday_flag    0
temperature     0
fuel_Price      0
cpi             0
unemployment    0
dtype: int64


In [122]:
# Convert date to numerical features
data['day'] = data['date'].dt.day
data['month'] = data['date'].dt.month
data['year'] = data['date'].dt.year

# # Drop unnecessary columns
data = data.drop(['date'], axis=1)

## data splitting

In [123]:
# Splitting features and target variable
X = data.drop(['weekly_sales'], axis=1)
y = data['weekly_sales']

# Splitting into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Train Model

In [124]:
# Train a Random Forest Regressor
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

## Evaluating the Model

In [134]:
# Predict on the test set
y_pred = model.predict(X_test)

# Calculate RMSE
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f'Root Mean Squared Error (RMSE): {rmse:.2f}')




Root Mean Squared Error (RMSE): 126105.33


## Examlpe of Prediction

In [126]:
i = 4
y_test.iloc[i],y_pred[i]

(428851.99, 441974.51190000045)

## Predict wekly sales for new example

In [127]:
# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def Predict(store, holiday_flag, temperature, fuel_Price, cpi, unemployment, date):
    """
    Predict weekly_sales using the given input features.

    Parameters:
    - store (int): Store identifier.
    - holiday_flag (int): Flag indicating whether it's a holiday (0 or 1).
    - temperature (float): Temperature in Fahrenheit.
    - fuel_Price (float): Fuel price.
    - cpi (float): Prevailing consumer price index.
    - unemployment (float): Prevailing unemployment rate.
    - date (str): Date in the format "YYYY-MM-DD".

    Returns:
    - Predicted weekly sales as a float.

    Example:
    predicted_sales = Predict(1, 0, 42, 2.561, 221.24, 8.10, "2023-01-12")
    """
    # Convert date to numerical features
    date_format = "%Y-%m-%d"  # The format of your date string
    try:
        date_object = datetime.strptime(date, date_format).date()

        # Create a dictionary to hold the function parameters
        data = {
            'store': [store],
            'holiday_flag': [holiday_flag],
            'temperature': [temperature],
            'fuel_Price': [fuel_Price],
            'cpi': [cpi],
            'unemployment': [unemployment]
        }

        # Create a DataFrame from the dictionary
        df = pd.DataFrame(data)

        df['day'] = date_object.day
        df['month'] = date_object.month
        df['year'] = date_object.year

        logger.info("Predicting weekly sales...")

        # Predict on the input features
        y_pred = model.predict(df)
        return y_pred[0]  # Assuming y_pred is an array with a single value

    except ValueError as e:
        logger.error("Invalid date format or other error occurred: %s", str(e))
        return None

In [130]:
y_pred = Predict(store = 1,holiday_flag= 0,temperature = 46.63,
             fuel_Price = 2.561,cpi = 211.3196429,unemployment = 8.106,date = "2024-02-26")

print('y_pred: {}'.format(y_pred))

y_pred: 1442004.0033000023


## Other Metrics

In [137]:
#R-squared (Coefficient of Determination
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

R2 = r2_score(y_test, y_pred)
print('R-squared (Coefficient of Determination) is : {}'.format(R2))


mae = mean_absolute_error(y_test, y_pred)
print('Mean Absolute Error (MAE) is : {}'.format(mae))



R-squared (Coefficient of Determination) is : 0.9506368811219579
Mean Absolute Error (MAE) is : 65709.74677700078


## Reponse